## Componentes XGBoosting

In [29]:
import numpy as np
import pandas as pd
import joblib
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb
import os
import json
from pathlib import Path

def criar_dataset_simples(series, look_back=5, passo=1):
    """
    Criar dataset SIMPLES - apenas sequência temporal (igual ao LSTM)
    SEM features engineering
    """
    X, y = [], []
    for i in range(len(series) - look_back - passo + 1):
        # Apenas a sequência temporal
        sequencia = series[i:i + look_back]
        target = series[i + look_back + passo - 1]
        
        X.append(sequencia.flatten())
        y.append(target[0])  # target é array, pegar valor
    
    return np.array(X), np.array(y)

def treinar_xgboost_componente(nome_comp, passo=1, test_size=0.2, random_state=42):
    """
    Treinar modelo XGBoost para um componente específico
    Usando APENAS sequências temporais (sem features engineering)
    """
    print(f"🌲 TREINANDO XGBOOST SIMPLES - {nome_comp.upper()} para t+{passo}")
    print("=" * 60)
    
    # 1. Carregar dados
    print(f"📊 1. Carregando dados {nome_comp.upper()}...")
    comp_file = f"{nome_comp.upper()}_component.csv"
    if not Path(comp_file).exists():
        raise FileNotFoundError(f"Arquivo não encontrado: {comp_file}")
    
    df = pd.read_csv(comp_file)
    if nome_comp.upper() not in df.columns:
        raise ValueError(f"Coluna {nome_comp.upper()} não encontrada em {comp_file}")
    
    data = df[nome_comp.upper()].values.reshape(-1, 1)
    print(f"   ✅ Dados carregados: {len(data)} pontos")
    print(f"   📈 Range: [{data.min():.4f}, {data.max():.4f}]")
    print(f"   📊 Std: {np.std(data):.4f}")
    
    # 2. Normalização
    print(f"\n🔧 2. Aplicando normalização...")
    scaler = RobustScaler()
    data_scaled = scaler.fit_transform(data)
    print(f"   ✅ Scaler: RobustScaler")
    print(f"   📈 Range normalizado: [{data_scaled.min():.4f}, {data_scaled.max():.4f}]")
    
    # 3. Criar dataset
    print(f"\n📦 3. Criando dataset...")
    look_back = 5 if passo in [1, 5] else 10  # Regra original
    print(f"   📏 Look_back: {look_back}")
    print(f"   🎯 Passo de previsão: {passo}")
    
    X, y = criar_dataset_simples(data_scaled, look_back, passo)
    print(f"   ✅ Dataset criado:")
    print(f"      X shape: {X.shape} (apenas sequências temporais)")
    print(f"      y shape: {y.shape}")
    print(f"      Features: {X.shape[1]} (= look_back, SEM features engineering)")
    
    # 4. Divisão treino/teste
    print(f"\n✂️ 4. Dividindo dados...")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, shuffle=False
    )
    print(f"   ✅ Treino: {X_train.shape[0]} amostras")
    print(f"   ✅ Teste: {X_test.shape[0]} amostras")
    
    # 5. Configurar XGBoost
    print(f"\n🤖 5. Configurando XGBoost...")
    xgb_params = {
        'objective': 'reg:squarederror',
        'max_depth': 6,
        'learning_rate': 0.1,
        'n_estimators': 100,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'random_state': random_state,
        'n_jobs': -1
    }
    
    print(f"   📋 Parâmetros:")
    for param, valor in xgb_params.items():
        print(f"      {param}: {valor}")
    
    # 6. Treinar modelo
    print(f"\n🚀 6. Treinando modelo...")
    modelo = xgb.XGBRegressor(**xgb_params)
    modelo.fit(X_train, y_train)
    print(f"   ✅ Treinamento concluído!")
    
    # 7. Avaliar modelo
    print(f"\n📊 7. Avaliando performance...")
    
    # Previsões
    y_pred_train = modelo.predict(X_train)
    y_pred_test = modelo.predict(X_test)
    
    # Desnormalizar para métricas
    y_train_real = scaler.inverse_transform(y_train.reshape(-1, 1)).flatten()
    y_test_real = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
    y_pred_train_real = scaler.inverse_transform(y_pred_train.reshape(-1, 1)).flatten()
    y_pred_test_real = scaler.inverse_transform(y_pred_test.reshape(-1, 1)).flatten()
    
    # Métricas treino
    rmse_train = np.sqrt(mean_squared_error(y_train_real, y_pred_train_real))
    mae_train = mean_absolute_error(y_train_real, y_pred_train_real)
    
    # Métricas teste
    rmse_test = np.sqrt(mean_squared_error(y_test_real, y_pred_test_real))
    mae_test = mean_absolute_error(y_test_real, y_pred_test_real)
    
    print(f"   📈 TREINO:")
    print(f"      RMSE: {rmse_train:.4f}")
    print(f"      MAE:  {mae_train:.4f}")
    print(f"   📈 TESTE:")
    print(f"      RMSE: {rmse_test:.4f}")
    print(f"      MAE:  {mae_test:.4f}")
    
    # Verificar overfitting
    overfitting_ratio = rmse_test / rmse_train
    if overfitting_ratio > 1.5:
        print(f"   ⚠️ Possível overfitting (ratio: {overfitting_ratio:.2f})")
    else:
        print(f"   ✅ Sem overfitting (ratio: {overfitting_ratio:.2f})")
    
    # 8. Salvar arquivos
    print(f"\n💾 8. Salvando arquivos...")
    
    # Criar diretórios
    os.makedirs("modelosXGB", exist_ok=True)
    os.makedirs("scalersXGB", exist_ok=True)
    os.makedirs("configsXGB", exist_ok=True)
    
    # Salvar modelo
    modelo_path = f"modelosXGB/xgb_{nome_comp.lower()}_t{passo}.joblib"
    joblib.dump(modelo, modelo_path)
    print(f"   ✅ Modelo salvo: {modelo_path}")
    
    # Salvar scaler
    scaler_path = f"scalersXGB/scaler_{nome_comp.lower()}_t{passo}.joblib"
    joblib.dump(scaler, scaler_path)
    print(f"   ✅ Scaler salvo: {scaler_path}")
    
    # Salvar configuração
    config = {
        'componente': nome_comp.upper(),
        'passo': passo,
        'look_back': look_back,
        'n_features_total': X.shape[1],
        'n_features_sequencia': look_back,
        'n_features_engineering': 0,  # SEM features engineering
        'scaler_type': 'RobustScaler',
        'model_type': 'XGBRegressor',
        'xgb_params': xgb_params,
        'dataset_info': {
            'total_samples': len(X),
            'train_samples': len(X_train),
            'test_samples': len(X_test),
            'data_range_original': [float(data.min()), float(data.max())],
            'data_range_scaled': [float(data_scaled.min()), float(data_scaled.max())]
        },
        'performance': {
            'rmse_train': float(rmse_train),
            'mae_train': float(mae_train),
            'rmse_test': float(rmse_test),
            'mae_test': float(mae_test),
            'overfitting_ratio': float(overfitting_ratio)
        }
    }
    
    config_path = f"configsXGB/config_{nome_comp.lower()}_t{passo}.json"
    with open(config_path, 'w', encoding='utf-8') as f:
        json.dump(config, f, indent=2, ensure_ascii=False)
    print(f"   ✅ Config salva: {config_path}")
    
    # 9. Teste de sanidade
    print(f"\n🧪 9. Teste de sanidade...")
    
    # Carregar modelo salvo e testar
    modelo_carregado = joblib.load(modelo_path)
    scaler_carregado = joblib.load(scaler_path)
    
    # Teste com uma amostra
    amostra_teste = X_test[:5]  # 5 amostras
    pred_teste = modelo_carregado.predict(amostra_teste)
    pred_teste_real = scaler_carregado.inverse_transform(pred_teste.reshape(-1, 1)).flatten()
    
    print(f"   ✅ Modelo carregado e testado:")
    print(f"      Amostras teste: {len(amostra_teste)}")
    print(f"      Previsões (normalizado): {pred_teste[:3]}")
    print(f"      Previsões (real): {pred_teste_real[:3]}")
    print(f"      Variabilidade: {np.std(pred_teste_real):.4f}")
    
    print(f"\n🎉 TREINAMENTO CONCLUÍDO COM SUCESSO!")
    print(f"   📁 Arquivos salvos:")
    print(f"      - {modelo_path}")
    print(f"      - {scaler_path}")
    print(f"      - {config_path}")
    
    return {
        'modelo': modelo,
        'scaler': scaler,
        'config': config,
        'performance': {
            'rmse_test': rmse_test,
            'mae_test': mae_test,
            'overfitting_ratio': overfitting_ratio
        }
    }

def treinar_todos_componentes(passos=[1, 5, 7, 30]):
    """
    Treinar modelos XGBoost para todos os componentes D1, D2, D3
    em todos os horizontes de previsão
    """
    print("🚀 TREINAMENTO COMPLETO - XGBOOST SIMPLES")
    print("=" * 70)
    print(f"📋 Componentes: D1, D2, D3")
    print(f"🎯 Horizontes: {passos}")
    print(f"🔧 Abordagem: Apenas sequências temporais (sem features)")
    print("=" * 70)
    
    resultados = {}
    componentes = ['d1', 'd2', 'd3']
    
    total_modelos = len(componentes) * len(passos)
    contador = 0
    
    for comp in componentes:
        resultados[comp] = {}
        
        for passo in passos:
            contador += 1
            print(f"\n{'🟢' * contador}{'⚪' * (total_modelos - contador)} PROGRESSO: {contador}/{total_modelos}")
            print(f"🎯 Treinando {comp.upper()} para t+{passo}")
            
            try:
                resultado = treinar_xgboost_componente(comp, passo)
                resultados[comp][f't+{passo}'] = resultado['performance']
                print(f"✅ {comp.upper()} t+{passo} - SUCESSO (RMSE: {resultado['performance']['rmse_test']:.4f})")
                
            except Exception as e:
                print(f"❌ {comp.upper()} t+{passo} - ERRO: {e}")
                resultados[comp][f't+{passo}'] = {'erro': str(e)}
    
    # Resumo final
    print(f"\n{'=' * 70}")
    print("📊 RESUMO FINAL DO TREINAMENTO")
    print("=" * 70)
    
    print(f"{'Componente':<12} {'Horizonte':<10} {'RMSE':<8} {'MAE':<8} {'Status':<10}")
    print("-" * 60)
    
    sucessos = 0
    falhas = 0
    
    for comp in componentes:
        for passo in passos:
            key = f't+{passo}'
            if key in resultados[comp]:
                if 'erro' in resultados[comp][key]:
                    print(f"{comp.upper():<12} {key:<10} {'N/A':<8} {'N/A':<8} {'ERRO':<10}")
                    falhas += 1
                else:
                    perf = resultados[comp][key]
                    rmse = perf['rmse_test']
                    mae = perf['mae_test']
                    print(f"{comp.upper():<12} {key:<10} {rmse:<8.4f} {mae:<8.4f} {'OK':<10}")
                    sucessos += 1
    
    print(f"\n📊 ESTATÍSTICAS:")
    print(f"   ✅ Sucessos: {sucessos}")
    print(f"   ❌ Falhas: {falhas}")
    print(f"   📈 Taxa de sucesso: {(sucessos/(sucessos+falhas)*100):.1f}%")
    
    # Salvar resumo
    resumo_path = "configsXGB/resumo_treinamento.json"
    resumo = {
        'data_treinamento': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
        'abordagem': 'XGBoost simples - apenas sequências temporais',
        'componentes': componentes,
        'horizontes': passos,
        'total_modelos': total_modelos,
        'sucessos': sucessos,
        'falhas': falhas,
        'resultados': resultados
    }
    
    with open(resumo_path, 'w', encoding='utf-8') as f:
        json.dump(resumo, f, indent=2, ensure_ascii=False)
    
    print(f"\n💾 Resumo salvo: {resumo_path}")
    
    if sucessos == total_modelos:
        print(f"\n🎉 TODOS OS MODELOS TREINADOS COM SUCESSO!")
    else:
        print(f"\n⚠️ Alguns modelos falharam. Verifique os logs acima.")
    
    return resultados

# Interface de uso
print("🌲 TREINAMENTO XGBOOST SIMPLES")
print("=" * 50)
print("📋 Funções disponíveis:")
print()
print("🔹 treinar_xgboost_componente('d1', passo=1)")
print("   → Treina um modelo específico")
print()
print("🔹 treinar_todos_componentes([1, 5, 7, 30])")
print("   → Treina todos os modelos para todos os horizontes")
print()
print("💡 CARACTERÍSTICAS DOS MODELOS:")
print("   • Apenas sequências temporais (sem features)")
print("   • Look_back: 5 para t+1,t+5 | 10 para demais")
print("   • Scaler: RobustScaler")
print("   • Modelo: XGBRegressor")
print("   • Consistente com LSTM (apenas sequências)")
print()
print("🎯 EXEMPLO DE USO:")
print("   # Treinar um modelo específico")
print("   resultado = treinar_xgboost_componente('d1', 1)")
print()
print("   # Treinar todos os modelos")
print("   resultados = treinar_todos_componentes([1, 5, 7, 30])")
print("=" * 50)

🌲 TREINAMENTO XGBOOST SIMPLES
📋 Funções disponíveis:

🔹 treinar_xgboost_componente('d1', passo=1)
   → Treina um modelo específico

🔹 treinar_todos_componentes([1, 5, 7, 30])
   → Treina todos os modelos para todos os horizontes

💡 CARACTERÍSTICAS DOS MODELOS:
   • Apenas sequências temporais (sem features)
   • Look_back: 5 para t+1,t+5 | 10 para demais
   • Scaler: RobustScaler
   • Modelo: XGBRegressor
   • Consistente com LSTM (apenas sequências)

🎯 EXEMPLO DE USO:
   # Treinar um modelo específico
   resultado = treinar_xgboost_componente('d1', 1)

   # Treinar todos os modelos
   resultados = treinar_todos_componentes([1, 5, 7, 30])


In [30]:
# Treinar todos os componentes D1, D2, D3 para todos os horizontes
resultados = treinar_todos_componentes([1, 5, 7, 30])

🚀 TREINAMENTO COMPLETO - XGBOOST SIMPLES
📋 Componentes: D1, D2, D3
🎯 Horizontes: [1, 5, 7, 30]
🔧 Abordagem: Apenas sequências temporais (sem features)

🟢⚪⚪⚪⚪⚪⚪⚪⚪⚪⚪⚪ PROGRESSO: 1/12
🎯 Treinando D1 para t+1
🌲 TREINANDO XGBOOST SIMPLES - D1 para t+1
📊 1. Carregando dados D1...
   ✅ Dados carregados: 7878 pontos
   📈 Range: [-7.7183, 8.8608]
   📊 Std: 1.3112

🔧 2. Aplicando normalização...
   ✅ Scaler: RobustScaler
   📈 Range normalizado: [-5.4508, 6.2790]

📦 3. Criando dataset...
   📏 Look_back: 5
   🎯 Passo de previsão: 1
   ✅ Dataset criado:
      X shape: (7873, 5) (apenas sequências temporais)
      y shape: (7873,)
      Features: 5 (= look_back, SEM features engineering)

✂️ 4. Dividindo dados...
   ✅ Treino: 6298 amostras
   ✅ Teste: 1575 amostras

🤖 5. Configurando XGBoost...
   📋 Parâmetros:
      objective: reg:squarederror
      max_depth: 6
      learning_rate: 0.1
      n_estimators: 100
      subsample: 0.8
      colsample_bytree: 0.8
      random_state: 42
      n_jobs: -1


## Varsão para mellhoara os picos e Depressões

In [31]:
# ====================================================================
# ENSEMBLE ESPECIALIZADO - IMPLEMENTAÇÃO PRÁTICA
# ====================================================================
# Este código SUBSTITUI o treinamento XGBoost normal
# Treina 3 modelos por componente: baixos, médios, altos

import numpy as np
import pandas as pd
import joblib
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb
import os
import json
from pathlib import Path

def criar_dataset_simples(series, look_back=5, passo=1):
    """Dataset simples - apenas sequências"""
    X, y = [], []
    for i in range(len(series) - look_back - passo + 1):
        sequencia = series[i:i + look_back]
        target = series[i + look_back + passo - 1]
        X.append(sequencia.flatten())
        y.append(target[0])
    return np.array(X), np.array(y)

def treinar_ensemble_especializado_componente(nome_comp, passo=1, test_size=0.2):
    """
    Treina ensemble especializado para um componente
    3 modelos: baixos, médios, altos + 1 modelo geral
    """
    print(f"🎯 ENSEMBLE ESPECIALIZADO - {nome_comp.upper()} t+{passo}")
    print("=" * 60)
    
    # 1. Carregar dados
    comp_file = f"{nome_comp.upper()}_component.csv"
    if not Path(comp_file).exists():
        raise FileNotFoundError(f"Arquivo não encontrado: {comp_file}")
    
    df = pd.read_csv(comp_file)
    data = df[nome_comp.upper()].values.reshape(-1, 1)
    
    print(f"📊 Dados carregados: {len(data)} pontos")
    print(f"   Range: [{data.min():.4f}, {data.max():.4f}]")
    
    # 2. Normalização
    scaler = RobustScaler()
    data_scaled = scaler.fit_transform(data)
    
    # 3. Criar dataset
    look_back = 5 if passo in [1, 5] else 10
    X, y = criar_dataset_simples(data_scaled, look_back, passo)
    
    print(f"📦 Dataset: {X.shape}, look_back={look_back}")
    
    # 4. Divisão treino/teste
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=42, shuffle=False
    )
    
    # 5. IDENTIFICAR REGIÕES (percentis na escala original)
    y_train_original = scaler.inverse_transform(y_train.reshape(-1, 1)).flatten()
    
    p25, p50, p75 = np.percentile(y_train_original, [25, 50, 75])
    print(f"🎯 Percentis: P25={p25:.3f}, P50={p50:.3f}, P75={p75:.3f}")
    
    # Máscaras para especialistas
    baixos_mask = y_train_original <= p25
    medios_mask = (y_train_original > p25) & (y_train_original < p75)
    altos_mask = y_train_original >= p75
    
    print(f"📊 Distribuição:")
    print(f"   Baixos: {np.sum(baixos_mask)} amostras ({np.sum(baixos_mask)/len(y_train)*100:.1f}%)")
    print(f"   Médios: {np.sum(medios_mask)} amostras ({np.sum(medios_mask)/len(y_train)*100:.1f}%)")
    print(f"   Altos:  {np.sum(altos_mask)} amostras ({np.sum(altos_mask)/len(y_train)*100:.1f}%)")
    
    # 6. TREINAR MODELOS ESPECIALISTAS
    modelos = {}
    
    # Parâmetros base
    base_params = {
        'random_state': 42,
        'n_jobs': -1,
        'verbosity': 0
    }
    
    # Modelo para BAIXOS (foco em depressões)
    if np.sum(baixos_mask) >= 20:  # Mínimo de amostras
        print(f"\n🔽 Treinando especialista BAIXOS...")
        modelo_baixos = xgb.XGBRegressor(
            max_depth=8,  # Mais profundo para capturar padrões raros
            learning_rate=0.05,  # Mais conservador
            n_estimators=300,  # Mais árvores
            subsample=0.9,
            colsample_bytree=0.9,
            objective='reg:absoluteerror',  # MAE para extremos
            **base_params
        )
        
        modelo_baixos.fit(X_train[baixos_mask], y_train[baixos_mask])
        
        # Avaliar
        if np.sum(baixos_mask) > 0:
            pred_baixos_train = modelo_baixos.predict(X_train[baixos_mask])
            rmse_baixos = np.sqrt(mean_squared_error(y_train[baixos_mask], pred_baixos_train))
            print(f"      RMSE treino: {rmse_baixos:.4f}")
        
        modelos['baixos'] = modelo_baixos
    else:
        print(f"⚠️ Poucos dados para especialista BAIXOS ({np.sum(baixos_mask)} amostras)")
        modelos['baixos'] = None
    
    # Modelo para ALTOS (foco em picos)
    if np.sum(altos_mask) >= 20:
        print(f"\n🔼 Treinando especialista ALTOS...")
        modelo_altos = xgb.XGBRegressor(
            max_depth=8,
            learning_rate=0.05,
            n_estimators=300,
            subsample=0.9,
            colsample_bytree=0.9,
            objective='reg:absoluteerror',
            **base_params
        )
        
        modelo_altos.fit(X_train[altos_mask], y_train[altos_mask])
        
        if np.sum(altos_mask) > 0:
            pred_altos_train = modelo_altos.predict(X_train[altos_mask])
            rmse_altos = np.sqrt(mean_squared_error(y_train[altos_mask], pred_altos_train))
            print(f"      RMSE treino: {rmse_altos:.4f}")
        
        modelos['altos'] = modelo_altos
    else:
        print(f"⚠️ Poucos dados para especialista ALTOS ({np.sum(altos_mask)} amostras)")
        modelos['altos'] = None
    
    # Modelo GERAL (baseline)
    print(f"\n⚖️ Treinando modelo GERAL...")
    modelo_geral = xgb.XGBRegressor(
        max_depth=6,  # Padrão
        learning_rate=0.1,
        n_estimators=200,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='reg:squarederror',  # RMSE padrão
        **base_params
    )
    
    modelo_geral.fit(X_train, y_train)
    modelos['geral'] = modelo_geral
    
    # 7. PREVISÃO ENSEMBLE NO TESTE
    print(f"\n🔀 Fazendo previsão ensemble...")
    
    pred_geral = modelo_geral.predict(X_test)
    pred_geral_original = scaler.inverse_transform(pred_geral.reshape(-1, 1)).flatten()
    
    # Identificar regiões no teste baseado na previsão geral
    p25_test, p75_test = np.percentile(pred_geral_original, [25, 75])
    
    baixos_test_mask = pred_geral_original <= p25_test
    altos_test_mask = pred_geral_original >= p75_test
    medios_test_mask = ~(baixos_test_mask | altos_test_mask)
    
    print(f"📊 Distribuição no teste:")
    print(f"   Baixos: {np.sum(baixos_test_mask)} ({np.sum(baixos_test_mask)/len(pred_geral)*100:.1f}%)")
    print(f"   Médios: {np.sum(medios_test_mask)} ({np.sum(medios_test_mask)/len(pred_geral)*100:.1f}%)")
    print(f"   Altos:  {np.sum(altos_test_mask)} ({np.sum(altos_test_mask)/len(pred_geral)*100:.1f}%)")
    
    # Previsão final combinada
    pred_final = pred_geral.copy()
    
    # Aplicar especialista BAIXOS
    if modelos['baixos'] is not None and np.sum(baixos_test_mask) > 0:
        pred_baixos = modelos['baixos'].predict(X_test[baixos_test_mask])
        # Combinar: 60% geral + 40% especialista
        pred_final[baixos_test_mask] = 0.6 * pred_final[baixos_test_mask] + 0.4 * pred_baixos
        print(f"   ✅ Aplicado especialista BAIXOS em {np.sum(baixos_test_mask)} pontos")
    
    # Aplicar especialista ALTOS
    if modelos['altos'] is not None and np.sum(altos_test_mask) > 0:
        pred_altos = modelos['altos'].predict(X_test[altos_test_mask])
        # Combinar: 60% geral + 40% especialista
        pred_final[altos_test_mask] = 0.6 * pred_final[altos_test_mask] + 0.4 * pred_altos
        print(f"   ✅ Aplicado especialista ALTOS em {np.sum(altos_test_mask)} pontos")
    
    # 8. AVALIAÇÃO
    print(f"\n📊 AVALIAÇÃO:")
    
    # Desnormalizar para métricas
    pred_geral_real = scaler.inverse_transform(pred_geral.reshape(-1, 1)).flatten()
    pred_final_real = scaler.inverse_transform(pred_final.reshape(-1, 1)).flatten()
    y_test_real = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
    
    # Métricas modelo geral
    rmse_geral = np.sqrt(mean_squared_error(y_test_real, pred_geral_real))
    mae_geral = mean_absolute_error(y_test_real, pred_geral_real)
    
    # Métricas ensemble
    rmse_ensemble = np.sqrt(mean_squared_error(y_test_real, pred_final_real))
    mae_ensemble = mean_absolute_error(y_test_real, pred_final_real)
    
    print(f"   🔘 Modelo GERAL:")
    print(f"      RMSE: {rmse_geral:.4f} | MAE: {mae_geral:.4f}")
    print(f"   🎯 ENSEMBLE:")
    print(f"      RMSE: {rmse_ensemble:.4f} | MAE: {mae_ensemble:.4f}")
    
    # Melhoria
    melhoria_rmse = ((rmse_geral - rmse_ensemble) / rmse_geral) * 100
    melhoria_mae = ((mae_geral - mae_ensemble) / mae_geral) * 100
    
    print(f"   🚀 MELHORIA:")
    print(f"      RMSE: {melhoria_rmse:+.2f}% | MAE: {melhoria_mae:+.2f}%")
    
    # 9. SALVAR MODELOS
    print(f"\n💾 Salvando modelos...")
    
    os.makedirs("modelosXGB_ensemble", exist_ok=True)
    os.makedirs("scalersXGB", exist_ok=True)
    os.makedirs("configsXGB", exist_ok=True)
    
    # Salvar ensemble completo
    ensemble_data = {
        'modelos': modelos,
        'scaler': scaler,
        'percentis_treino': {'p25': p25, 'p50': p50, 'p75': p75},
        'pesos_combinacao': {'geral': 0.6, 'especialista': 0.4}
    }
    
    ensemble_path = f"modelosXGB_ensemble/ensemble_{nome_comp.lower()}_t{passo}.joblib"
    joblib.dump(ensemble_data, ensemble_path)
    
    # Salvar scaler (compatibilidade)
    scaler_path = f"scalersXGB/scaler_{nome_comp.lower()}_t{passo}.joblib"
    joblib.dump(scaler, scaler_path)
    
    # Salvar config
    config = {
        'componente': nome_comp.upper(),
        'passo': passo,
        'tipo': 'ensemble_especializado',
        'look_back': look_back,
        'n_features_total': X.shape[1],
        'modelos_treinados': list(modelos.keys()),
        'performance': {
            'rmse_geral': float(rmse_geral),
            'mae_geral': float(mae_geral),
            'rmse_ensemble': float(rmse_ensemble),
            'mae_ensemble': float(mae_ensemble),
            'melhoria_rmse_pct': float(melhoria_rmse),
            'melhoria_mae_pct': float(melhoria_mae)
        }
    }
    
    config_path = f"configsXGB/config_{nome_comp.lower()}_t{passo}.json"
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=2)
    
    print(f"   ✅ Ensemble salvo: {ensemble_path}")
    print(f"   ✅ Config salva: {config_path}")
    
    return ensemble_data

def treinar_todos_ensembles(passos=[1, 5, 7, 30]):
    """
    Treina ensembles especializados para todos os componentes e horizontes
    """
    print("🚀 TREINAMENTO ENSEMBLE ESPECIALIZADO - TODOS OS MODELOS")
    print("=" * 70)
    
    componentes = ['d1', 'd2', 'd3']
    total_modelos = len(componentes) * len(passos)
    contador = 0
    
    resultados = {}
    
    for comp in componentes:
        resultados[comp] = {}
        
        for passo in passos:
            contador += 1
            print(f"\n{'🟢' * contador}{'⚪' * (total_modelos - contador)} PROGRESSO: {contador}/{total_modelos}")
            
            try:
                ensemble_data = treinar_ensemble_especializado_componente(comp, passo)
                
                # Extrair métricas
                config_path = f"configsXGB/config_{comp}_t{passo}.json"
                with open(config_path, 'r') as f:
                    config = json.load(f)
                
                resultados[comp][f't+{passo}'] = config['performance']
                
                print(f"✅ {comp.upper()} t+{passo} - Melhoria RMSE: {config['performance']['melhoria_rmse_pct']:+.2f}%")
                
            except Exception as e:
                print(f"❌ {comp.upper()} t+{passo} - ERRO: {e}")
                resultados[comp][f't+{passo}'] = {'erro': str(e)}
    
    # Resumo final
    print(f"\n{'=' * 70}")
    print("📊 RESUMO FINAL - ENSEMBLE ESPECIALIZADO")
    print("=" * 70)
    
    melhorias_rmse = []
    melhorias_mae = []
    
    for comp in componentes:
        for passo in passos:
            key = f't+{passo}'
            if key in resultados[comp] and 'melhoria_rmse_pct' in resultados[comp][key]:
                perf = resultados[comp][key]
                print(f"{comp.upper()} {key}: RMSE {perf['melhoria_rmse_pct']:+.1f}% | MAE {perf['melhoria_mae_pct']:+.1f}%")
                melhorias_rmse.append(perf['melhoria_rmse_pct'])
                melhorias_mae.append(perf['melhoria_mae_pct'])
    
    if melhorias_rmse:
        print(f"\n📈 MELHORIA MÉDIA:")
        print(f"   RMSE: {np.mean(melhorias_rmse):+.2f}% ± {np.std(melhorias_rmse):.2f}%")
        print(f"   MAE:  {np.mean(melhorias_mae):+.2f}% ± {np.std(melhorias_mae):.2f}%")
    
    return resultados

# Interface de uso
print("🎯 ENSEMBLE ESPECIALIZADO - IMPLEMENTAÇÃO PRÁTICA")
print("=" * 60)
print("📋 FUNÇÕES DISPONÍVEIS:")
print()
print("🔹 treinar_ensemble_especializado_componente('d1', passo=1)")
print("   → Treina ensemble para um componente específico")
print()
print("🔹 treinar_todos_ensembles([1, 5, 7, 30])")
print("   → Treina ensembles para todos os componentes")
print()
print("💡 COMO USAR:")
print("   1. Execute: treinar_todos_ensembles([1, 5, 7, 30])")
print("   2. Aguarde o treinamento (5-15 minutos)")
print("   3. Use o código de avaliação normalmente")
print()
print("🎯 DIFERENÇA:")
print("   • 3 modelos por componente (baixos/médios/altos)")
print("   • Foco específico em picos e depressões")
print("   • Combinação inteligente das previsões")
print("=" * 60)

🎯 ENSEMBLE ESPECIALIZADO - IMPLEMENTAÇÃO PRÁTICA
📋 FUNÇÕES DISPONÍVEIS:

🔹 treinar_ensemble_especializado_componente('d1', passo=1)
   → Treina ensemble para um componente específico

🔹 treinar_todos_ensembles([1, 5, 7, 30])
   → Treina ensembles para todos os componentes

💡 COMO USAR:
   1. Execute: treinar_todos_ensembles([1, 5, 7, 30])
   2. Aguarde o treinamento (5-15 minutos)
   3. Use o código de avaliação normalmente

🎯 DIFERENÇA:
   • 3 modelos por componente (baixos/médios/altos)
   • Foco específico em picos e depressões
   • Combinação inteligente das previsões


In [32]:
# Treinar todos os modelos ensemble
resultados = treinar_todos_ensembles([1, 5, 7, 30])

🚀 TREINAMENTO ENSEMBLE ESPECIALIZADO - TODOS OS MODELOS

🟢⚪⚪⚪⚪⚪⚪⚪⚪⚪⚪⚪ PROGRESSO: 1/12
🎯 ENSEMBLE ESPECIALIZADO - D1 t+1
📊 Dados carregados: 7878 pontos
   Range: [-7.7183, 8.8608]
📦 Dataset: (7873, 5), look_back=5
🎯 Percentis: P25=-0.755, P50=-0.021, P75=0.748
📊 Distribuição:
   Baixos: 1575 amostras (25.0%)
   Médios: 3148 amostras (50.0%)
   Altos:  1575 amostras (25.0%)

🔽 Treinando especialista BAIXOS...
      RMSE treino: 0.2174

🔼 Treinando especialista ALTOS...
      RMSE treino: 0.2179

⚖️ Treinando modelo GERAL...

🔀 Fazendo previsão ensemble...
📊 Distribuição no teste:
   Baixos: 394 (25.0%)
   Médios: 787 (50.0%)
   Altos:  394 (25.0%)
   ✅ Aplicado especialista BAIXOS em 394 pontos
   ✅ Aplicado especialista ALTOS em 394 pontos

📊 AVALIAÇÃO:
   🔘 Modelo GERAL:
      RMSE: 0.6245 | MAE: 0.3948
   🎯 ENSEMBLE:
      RMSE: 0.6357 | MAE: 0.4048
   🚀 MELHORIA:
      RMSE: -1.80% | MAE: -2.53%

💾 Salvando modelos...
   ✅ Ensemble salvo: modelosXGB_ensemble/ensemble_d1_t1.joblib
  